Stock Neo4j Northwind example.

 - Upload the provided example csv files to the free neo4j instance
 - Use the web app wizard to prototype Order-Product relationships
 - Load the entire graph using the map function
 - Examine the graph via Langchain neo4j connector

In [1]:
import os
from dotenv import load_dotenv

#openai
import openai

# langchain
from langchain.chat_models import ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [2]:
graph = Neo4jGraph(
    url="neo4j+s://67f780a0.databases.neo4j.io",
    username=os.getenv('NEO4J_USERNAME'),
    password=os.getenv('NEO4J_PS')
)

In [3]:
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph, verbose=True
)

In [4]:
chain.run('what product was in the most orders')



> Entering new  chain...
Generated Cypher:
MATCH (o:Order)-[:CONTAINS]->(p:Product)
RETURN p.productName, COUNT(o) AS orderCount
ORDER BY orderCount DESC
LIMIT 1
Full Context:
[{'p.productName': 'Raclette Courdavault', 'orderCount': 54}]

> Finished chain.


'The product that was in the most orders is Raclette Courdavault.'

In [5]:
chain.run('what orders contained coffee')



> Entering new  chain...
Generated Cypher:
MATCH (o:Order)-[:CONTAINS]->(p:Product)
WHERE p.productName = 'coffee'
RETURN o
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have any information on what orders contained coffee."

In [6]:
chain.run('how many orders had at least three products in common')



> Entering new  chain...
Generated Cypher:
MATCH (o1:Order)-[:CONTAINS]->(p1:Product)
MATCH (o2:Order)-[:CONTAINS]->(p2:Product)
WHERE o1 <> o2 AND p1 = p2
WITH o1, o2, COUNT(DISTINCT p1) AS commonProducts
WHERE commonProducts >= 3
RETURN COUNT(DISTINCT o1) AS numberOfOrders
Full Context:
[{'numberOfOrders': 44}]

> Finished chain.


"I'm sorry, but I don't have enough information to answer your question."

In [7]:
chain.run('what was included in the most recent order. Also include the orderid')



> Entering new  chain...
Generated Cypher:
MATCH (o:Order)-[:CONTAINS]->(p:Product)
WITH o, p
ORDER BY o.orderDate DESC
LIMIT 1
RETURN p.productName, o.id AS orderid
Full Context:
[{'p.productName': 'Pavlova', 'orderid': '11074'}]

> Finished chain.


'The most recent order included the product "Pavlova" and the order ID is 11074.'

In [8]:
chain.run('who made the most orders?')



> Entering new  chain...
Generated Cypher:
MATCH (o:Order)-[:CONTAINS]->(p:Product)
WITH o, COUNT(o) AS orderCount
RETURN o.id AS customerId, orderCount
ORDER BY orderCount DESC
LIMIT 1
Full Context:
[{'customerId': '11077', 'orderCount': 25}]

> Finished chain.


'The customer with the ID 11077 made the most orders, with a total of 25 orders.'

In [9]:
chain.run('what product was ordered the least times?')



> Entering new  chain...
Generated Cypher:
MATCH (o:Order)-[:CONTAINS]->(p:Product)
RETURN p.productName, COUNT(o) AS orderCount
ORDER BY orderCount ASC
LIMIT 1
Full Context:
[{'p.productName': 'Mishi Kobe Niku', 'orderCount': 5}]

> Finished chain.


'The product that was ordered the least times is Mishi Kobe Niku, with a total of 5 orders.'